<a href="https://colab.research.google.com/github/calixphd/Quantum-Simulation-Tutorials-and-Examples/blob/main/Introduction_to_Quantum_Generative_Adversarial_Networks_Theory_with_Cirq_%26_TensorFlow_Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pennylane

In [22]:
! pip install pennylane-cirq

# Introduction to Quantum Generative Adversarial Networks: Theory with Cirq & TensorFlow Implementation

# Introduction

We present two implementations  a Quantum Generative Adversarial Network (QGAN) (Lloyd and Weedbrook (2018), Dallaire-Demers and Killoran (2018)). The first implementation 


 using two subcircuits, a generator and a discriminator. The generator attempts to generate synthetic quantum data to match a pattern of “real” data, while the discriminator tries to discern real data from fake data. The gradient of the discriminator’s output provides a training signal for the generator to improve its fake generated data.

## We begin by importing PennyLane, NumPy, and TensorFlow.

We begin by importing PennyLane, NumPy, and TensorFlow.

In [23]:
import pennylane as qml
import numpy as np
import tensorflow as tf

Declare a 3-qubit simulator device on Cirq.

In [24]:
dev = qml.device('cirq.simulator', wires=3, shots=1000)

# The Adversaries: Generator and Discriminator

The usual starting point is to draw samples either from some “real data” distribution, or from the generator, and feed them to the discriminator. In this tutorial we will use a quantum circuit to generate the real data.

For this simple example, our real data will be a qubit that has been rotated (from the starting state |0⟩) to some arbitrary, but fixed, state.

In [25]:
def real(angles, **kwargs):
    qml.Hadamard(wires=0)
    qml.Rot(*angles, wires=0)

For the generator and discriminator, we will choose the same basic circuit structure, but acting on different wires.

Both the real data circuit and the generator will output on wire 0, which will be connected as an input to the discriminator. Wire 1 is provided as a workspace for the generator, while the discriminator’s output will be on wire 2.

In [26]:
def generator(w, **kwargs):
    qml.Hadamard(wires=0)
    qml.RX(w[0], wires=0)
    qml.RX(w[1], wires=1)
    qml.RY(w[2], wires=0)
    qml.RY(w[3], wires=1)
    qml.RZ(w[4], wires=0)
    qml.RZ(w[5], wires=1)
    qml.CNOT(wires=[0, 1])
    qml.RX(w[6], wires=0)
    qml.RY(w[7], wires=0)
    qml.RZ(w[8], wires=0)


def discriminator(w):
    qml.Hadamard(wires=0)
    qml.RX(w[0], wires=0)
    qml.RX(w[1], wires=2)
    qml.RY(w[2], wires=0)
    qml.RY(w[3], wires=2)
    qml.RZ(w[4], wires=0)
    qml.RZ(w[5], wires=2)
    qml.CNOT(wires=[0, 2])
    qml.RX(w[6], wires=2)
    qml.RY(w[7], wires=2)
    qml.RZ(w[8], wires=2)

We create two QNodes. One where the real data source is wired up to the discriminator, and one where the generator is connected to the discriminator. In order to pass TensorFlow Variables into the quantum circuits, we specify the "tf" interface.

In [27]:
@qml.qnode(dev, interface="tf")
def real_disc_circuit(phi, theta, omega, disc_weights):
    real([phi, theta, omega])
    discriminator(disc_weights)
    return qml.expval(qml.PauliZ(2))


@qml.qnode(dev, interface="tf")
def gen_disc_circuit(gen_weights, disc_weights):
    generator(gen_weights)
    discriminator(disc_weights)
    return qml.expval(qml.PauliZ(2))

# Cost Functions, Training & Optimization

There are two cost functions of interest, corresponding to the two stages of QGAN training. These cost functions are built from two pieces: the first piece is the probability that the discriminator correctly classifies real data as real. The second piece is the probability that the discriminator classifies fake data (i.e., a state prepared by the generator) as real.

The discriminator is trained to maximize the probability of correctly classifying real data, while minimizing the probability of mistakenly classifying fake data.

CostD=Pr(real|fake)−Pr(real|real)
The generator is trained to maximize the probability that the discriminator accepts fake data as real.

CostG=−Pr(real|fake)

In [34]:
def prob_real_true(disc_weights):
    true_disc_output = real_disc_circuit(phi, theta, omega, disc_weights)
    # convert to probability
    prob_real_true = (true_disc_output + 1) / 2
    return prob_real_true


def prob_fake_true(gen_weights, disc_weights):
    fake_disc_output = gen_disc_circuit(gen_weights, disc_weights)
    # convert to probability
    prob_fake_true = (fake_disc_output + 1) / 2
    return prob_fake_true


def disc_cost(disc_weights):
    cost = prob_fake_true(gen_weights, disc_weights) - prob_real_true(disc_weights)
    return cost


def gen_cost(gen_weights):
    return -prob_fake_true(gen_weights, disc_weights)

### The Training Phase

We initialize the fixed angles of the “real data” circuit, as well as the initial parameters for both generator and discriminator. These are chosen so that the generator initially prepares a state on wire 0 that is very close to the |1⟩ state.

In [38]:
phi = np.pi / 6
theta = np.pi / 2
omega = np.pi / 7
np.random.seed(0)
eps = 1e-2
init_gen_weights = np.array([np.pi] + [0] * 8) + \
                   np.random.normal(scale=eps, size=(9,))
init_disc_weights = np.random.normal(size=(9,))

gen_weights = tf.Variable(init_gen_weights)
disc_weights = tf.Variable(init_disc_weights)

### Create an optimizer

In [39]:
opt = tf.keras.optimizers.SGD(0.4)

### First, optimize the discriminator while keeping the generator parameters fixed.

In [40]:
cost = lambda: disc_cost(disc_weights)

for step in range(50):
    opt.minimize(cost, disc_weights)
    if step % 5 == 0:
        cost_val = cost().numpy()
        print("Step {}: cost = {}".format(step, cost_val))

Step 0: cost = -0.07400000000000001
Step 5: cost = -0.27299999999999996
Step 10: cost = -0.445
Step 15: cost = -0.4780000000000001
Step 20: cost = -0.46799999999999997
Step 25: cost = -0.498
Step 30: cost = -0.512
Step 35: cost = -0.492
Step 40: cost = -0.495
Step 45: cost = -0.496


As the discriminator becomes optimal, the probability for the discriminator to correctly classify the real data should becomes close to one.

In [41]:
print("Prob(real classified as real): ", prob_real_true(disc_weights).numpy())

Prob(real classified as real):  0.997


### Sanity check 

How does the discriminator classify the generator’s (still unoptimized) fake data:



In [42]:
print("Prob(fake classified as real): ", prob_fake_true(gen_weights, disc_weights).numpy())

Prob(fake classified as real):  0.505


### Fooling the discriminator
To play the adversarial game, train the generator to better fool the discriminator. For this simple tutorial, we only perform one stage of the game. For more complex models, we would continue training the models in an alternating fashion until we reach the optimum point of the two-player adversarial game.

In [43]:
cost = lambda: gen_cost(gen_weights)

for step in range(50):
    opt.minimize(cost, gen_weights)
    if step % 5 == 0:
        cost_val = cost().numpy()
        print("Step {}: cost = {}".format(step, cost_val))

Step 0: cost = -0.605
Step 5: cost = -0.884
Step 10: cost = -0.982
Step 15: cost = -0.994
Step 20: cost = -0.999
Step 25: cost = -0.999
Step 30: cost = -1.0
Step 35: cost = -1.0
Step 40: cost = -1.0
Step 45: cost = -1.0


At the optimum of the generator, the probability for the discriminator to be fooled should be close to 1.

In [44]:
print("Prob(fake classified as real): ", prob_fake_true(gen_weights, disc_weights).numpy())

Prob(fake classified as real):  1.0


As the two adversaries approach optimum, the discriminator cost will be close to zero, indicating that the discriminator assigns equal probability to both real and generated data indicating that the generator has successfully learned how to simulate the real data enough to fool the discriminator.

In [45]:
print("Discriminator cost: ", disc_cost(disc_weights).numpy())

Discriminator cost:  0.0030000000000000027


# Conclusion

Conclude by comparing the states of the real data circuit and the generator. Observe that the generator have learned to be in a state that is very close to the one prepared in the real data circuit. The state of the first qubit is represented through its Bloch sphere representation as follows

In [46]:
obs = [qml.PauliX(0), qml.PauliY(0), qml.PauliZ(0)]
bloch_vector_real = qml.map(real, obs, dev, interface="tf")
bloch_vector_generator = qml.map(generator, obs, dev, interface="tf")

print("Real Bloch vector: {}".format(bloch_vector_real([phi, theta, omega])))
print("Generator Bloch vector: {}".format(bloch_vector_generator(gen_weights)))

Real Bloch vector: [-0.166  0.422 -0.86 ]
Generator Bloch vector: [-0.28   0.424 -0.86 ]


# **References**

